# Clustering Case Study 2: Apply Association Rules to the customer segments from Case Study 1 to create a recommendation engine 

## Overview of Association Rules and the Apriori algorithm behind it 

Association Rules uncovers which items in a dataset occur together. Within the context of our ecommerce dataset, if customers normally purchase 

KDNuggets gives a quick overview [here](https://www.kdnuggets.com/2016/04/association-rules-apriori-algorithm-tutorial.html). For a more mathematical overview, see [pg 497 of ESL by Hastie and Tibshirani](https://web.stanford.edu/~hastie/Papers/ESLII.pdf) 

Association Rules are particularly useful for stock transaction data and provide a good starting point into recommendation engines. 

## Implementing Association Rules on ecommerce data 

1. Read in the cleaned dataset you saved in Case Study 1
2. This dataset is not ready for Association Rules yet. Therefore, reshape the data so that each row is an invoice number and each column is a product
![alt text](stockcode.png)

In [1]:
# %load src/dataCleaning.py
import pandas as pd
import numpy as np

def dropEntryWithoutCustomerID(df_input):
    df_output = df_input.dropna(axis=0, subset=['CustomerID'])
    return df_output

def dropDuplicatedEntries(df_input):
    df_output = df_input.drop_duplicates()
    return df_output

def removeQuatityLessThanZero(df):
	temp = df[df.Quantity > 0]
	return temp

def featureEngineer(temp):
    intermediate = temp.groupby(['CustomerID']).agg({'InvoiceNo': "nunique",
                                         'StockCode': "nunique",
                                          'Quantity': ["sum",'count'],
                                          'UnitPrice': ['mean','std']})
    
    # Using ravel, and a string join, we can create better names for the columns:
    intermediate.columns = ["_".join(x) for x in intermediate.columns.ravel()]
    
    intermediate['QuantityPerInvoice'] = intermediate['Quantity_sum']/intermediate['InvoiceNo_nunique']
    intermediate['UniqueItemsPerInvoice'] = intermediate['Quantity_count']/intermediate['InvoiceNo_nunique']
    
    intermediate.drop(['Quantity_sum'], axis=1, inplace=True)
    
    intermediate.rename(columns={'InvoiceNo_nunique':'NoOfInvoices','StockCode_nunique':'NoOfUniqueItems',
                             'Quantity_count':'TotalQuantity','UnitPrice_mean':'UnitPriceMean',
                            'UnitPrice_std':'UnitPriceStd'}, inplace=True)
    
    intermediate.fillna(0, inplace=True)
    
    return intermediate

def cleanAndEngineer(df):
	cleanedData = featureEngineer(removeQuatityLessThanZero(dropDuplicatedEntries(dropEntryWithoutCustomerID(df))))

	return cleanedData

In [10]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_samples, silhouette_score
from scipy import stats

In [3]:
df_raw = pd.read_csv('data/raw/data.csv', encoding='ISO-8859-1')

df = removeQuatityLessThanZero(dropDuplicatedEntries(dropEntryWithoutCustomerID(df_raw)))

df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [36]:
data = pd.crosstab(df.InvoiceNo, df.StockCode)
data.head()

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


3. Apply the apriori algorithm on the dataset generated above to get the frequent itemsets. You may find the `mlextend` libary useful
4. Apply association rules on the frequent itemsets from 3 to generate confidence, support and lift measures for the data 
5. What happens when you change the `min_threshold` parameter? 

In [5]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

data_sets = data.applymap(encode_units)

In [6]:
frequent_itemsets = apriori(data_sets, min_support=0.01, use_colnames=True)

In [46]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=20)
rules.sort_values(by=['lift'], ascending=False).head(n=5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
41,(23172),(23171),0.012085,0.014566,0.010898,0.901786,61.909259,0.010722,10.033507
40,(23171),(23172),0.014566,0.012085,0.010898,0.748148,61.909259,0.010722,3.922605
89,(22746),"(22745, 22748)",0.013595,0.013703,0.010035,0.738095,53.863517,0.009848,3.765861
88,"(22745, 22748)",(22746),0.013703,0.013595,0.010035,0.732283,53.863517,0.009848,3.684512
47,(23175),(23174),0.014674,0.014458,0.011114,0.757353,52.381694,0.010901,4.061626


When min_threshold parameter is changed, the rules are filtered based on the metric of interest to only show the rules above the stated minimum threshold.

### Creating tailored recommendations by applying Association Rules to the customer segments produced from Case Study 1

1. In the previous notebook, we created a GMM model that clustered customers into n segments. Apply association rules to each segment from your chosen model. 
2. Do results for each segment differ from each other? 

In [14]:
intermediate = featureEngineer(df)
scaler = StandardScaler()
intermediate_scaled = scaler.fit_transform(intermediate)

# Reset the CustomerID index back into normal column
intermediate_reset = intermediate.reset_index(level=0, inplace=False)

gmm = GaussianMixture(n_components=5,random_state =7)
gmm.fit(intermediate_scaled)
gmmlabels = gmm.predict(intermediate_scaled)

gmmlabels_df = pd.DataFrame(gmmlabels)
gmm_outcome = pd.concat([intermediate_reset, gmmlabels_df], axis=1)

gmm_outcome.rename(columns={0:'Clusters'}, inplace=True)

gmm_outcome.groupby(by=['Clusters'])['CustomerID'].count()

Clusters
0    2798
1     260
2    1276
3       4
4       1
Name: CustomerID, dtype: int64

In [15]:
gmm_outcome.head()

,CustomerID,NoOfInvoices,NoOfUniqueItems,TotalQuantity,UnitPriceMean,UnitPriceStd,QuantityPerInvoice,UniqueItemsPerInvoice,Clusters
0,12346.0,1,1,1,1.040000,0.000000,74215.000000,1.00,2
1,12347.0,7,103,182,2.644011,2.255381,351.142857,26.00,2
2,12348.0,4,22,31,5.764839,13.400323,585.250000,7.75,1
3,12349.0,1,73,73,8.289041,35.028021,631.000000,73.00,1
4,12350.0,1,17,17,3.841176,9.334751,197.000000,17.00,0


Will apply the association rules to only cluster 0, 1 and 2. Cluster 3 and 4 would be omitted because they are too small. These two clusters could have a customised business plan to target them since they are very unique customers. 

In [29]:
gmm_outcome_0 = gmm_outcome[gmm_outcome['Clusters'] == 0]['CustomerID']
gmm_outcome_1 = gmm_outcome[gmm_outcome['Clusters'] == 1]['CustomerID']
gmm_outcome_2 = gmm_outcome[gmm_outcome['Clusters'] == 2]['CustomerID']

print("Number of customer in cluster 0: ", gmm_outcome_0.count())
print("Number of customer in cluster 1: ", gmm_outcome_1.count())
print("Number of customer in cluster 2: ", gmm_outcome_2.count())

Number of customer in cluster 0:  2798
Number of customer in cluster 1:  260
Number of customer in cluster 2:  1276


In [30]:
df_0 = df[df['CustomerID'].isin(gmm_outcome_0)]
print("Number of transaction in cluster 0: ", df_0['CustomerID'].count())

Number of transaction in cluster 0:  89547


In [31]:
df_1 = df[df['CustomerID'].isin(gmm_outcome_1)]
print("Number of transaction in cluster 1: ", df_1['CustomerID'].count())

Number of transaction in cluster 1:  73835


In [32]:
df_2 = df[df['CustomerID'].isin(gmm_outcome_2)]
print("Number of transaction in cluster 2: ", df_2['CustomerID'].count())

Number of transaction in cluster 2:  206477


In [45]:
data_0 = pd.crosstab(df_0.InvoiceNo, df_0.StockCode)
data_sets_0 = data_0.applymap(encode_units)
frequent_itemsets_0 = apriori(data_sets_0, min_support=0.01, use_colnames=True)
rules_0 = association_rules(frequent_itemsets_0, metric="lift", min_threshold=20)
rules_0.sort_values(by=['lift'], ascending=False).head(n=5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
21,(23256),(23254),0.012958,0.012440,0.010194,0.786667,63.239259,0.010032,4.629190
20,(23254),(23256),0.012440,0.012958,0.010194,0.819444,63.239259,0.010032,5.466695
28,(47590A),(47590B),0.017104,0.017623,0.013303,0.777778,44.135076,0.013002,4.420698
29,(47590B),(47590A),0.017623,0.017104,0.013303,0.754902,44.135076,0.013002,4.010214
6,(22629),(22630),0.019350,0.018314,0.013131,0.678571,37.052561,0.012776,3.054135


In [43]:
data_1 = pd.crosstab(df_1.InvoiceNo, df_1.StockCode)
data_sets_1 = data_1.applymap(encode_units)
frequent_itemsets_1 = apriori(data_sets_1, min_support=0.01, use_colnames=True)
rules_1 = association_rules(frequent_itemsets_1, metric="lift", min_threshold=20)
rules_1.sort_values(by=['lift'], ascending=False).head(n=5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
40,(22917),(22918),0.011053,0.010526,0.010526,0.952381,90.476190,0.010410,20.778947
41,(22918),(22917),0.010526,0.011053,0.010526,1.000000,90.476190,0.010410,inf
42,(22917),(22920),0.011053,0.010263,0.010000,0.904762,88.156288,0.009887,10.392237
43,(22920),(22917),0.010263,0.011053,0.010000,0.974359,88.156288,0.009887,38.568947
69,(23286),(23287),0.013684,0.011579,0.010526,0.769231,66.433566,0.010368,4.283158


In [44]:
data_2 = pd.crosstab(df_2.InvoiceNo, df_2.StockCode)
data_sets_2 = data_2.applymap(encode_units)
frequent_itemsets_2 = apriori(data_sets_2, min_support=0.01, use_colnames=True)
rules_2 = association_rules(frequent_itemsets_2, metric="lift", min_threshold=20)
rules_2.sort_values(by=['lift'], ascending=False).head(n=5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
266,"(22917, 22918)",(22916),0.010363,0.011554,0.010006,0.965517,83.562033,0.009886,28.664920
271,(22916),"(22917, 22918)",0.011554,0.010363,0.010006,0.865979,83.562033,0.009886,7.384212
273,"(22919, 22916)",(22917),0.010482,0.011435,0.010006,0.954545,83.473011,0.009886,21.748422
276,(22917),"(22919, 22916)",0.011435,0.010482,0.010006,0.875000,83.473011,0.009886,7.916141
272,"(22919, 22917)",(22916),0.010482,0.011554,0.010006,0.954545,82.612465,0.009885,21.745801


From the association rule analysis, it can be seen that the 3 clusters produce very different association of itemsets. The top 5 antecedents and consequents combination for the 3 clusters are very different. This suggests that the clustering might be quite good and the business teams can make use of this information to tailor make marketing plans for each of the cluster.